# "Blight Model - Predicting Ticket Payment"
### Gradient Boost Classifier Project for Ann-Arbor Michigan
#### -- Assignment 4 for Applied Machine Learning - University of Michigan --
#### -- By John G. Crowley

In [22]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
#from sklearn.model_selection import GridSearchCV

y = 'readonly/addresses.csv'
z = 'readonly/latlons.csv'
w = 'readonly/train.csv'
x= 'readonly/test.csv'


def blight_model(w,x,y,z):
    
    def wrangle():
        train = pd.read_csv(w,engine='python')
        test = pd.read_csv(x)


        train['disposition'].replace(to_replace='Not responsible by City Dismissal',
                                    value='Not responsible by Dismissal',inplace=True)
        dispos_keep_list= ['Responsible by Default', 'Responsible by Determination',
                           'Responsible by Admission', 'Responsible (Fine Waived) by Deter']

        train = train[train['disposition'].isin(dispos_keep_list)].copy()

        dispo_dict = {'Responsible (Fine Waived) by Admis':'Responsible by Admission',
                      'Responsible - Compl/Adj by Default':'Responsible by Default',
                      'Responsible - Compl/Adj by Determi':'Responsible by Determination'}
        test['disposition'].replace(dispo_dict,inplace=True)

        #test = test[test['disposition'].isin(dispos_keep_list)].copy()

        test = test[test.columns[[0,18,25]]].copy()
        test = pd.get_dummies(test,columns=['disposition']).copy()

        #drop columns not needed after examining each
        idx = [1,2,3,6,7,8,9,10,11,12,13,14,15,16,17,28,30,31,32]
        train.drop(train.columns[idx],axis=1,inplace=True)

        idxes = [1,2,4,5,6,7,8,9,11,12]
        train.drop(train.columns[idxes],axis=1,inplace=True)

        train = pd.get_dummies(train,columns=['disposition']).copy()

        train.drop(train.columns[2],axis=1,inplace=True)


        def geo():
            addys = pd.read_csv(y)
            latlon = pd.read_csv(z)

            addr = addys['address'].str.extract('([0-9\sA-Za-z]+),',expand=False)
            addys['merge_addy'] = addr

            latlon['merge_addy'] = latlon['address'].copy()
            latlon['merge_addy'] = latlon['merge_addy'].str.replace(',.*','')

            latlon_addy = pd.merge(latlon,addys,how='inner',on='merge_addy')
            latlon_addy.drop(['address_x','address_y'],axis=1,inplace=True)
            latlon_addy.drop_duplicates(subset='ticket_id',keep='first',inplace=True)

            training = pd.merge(latlon_addy,train,how='inner',on='ticket_id').dropna()
            training.sort_values(by='ticket_id',inplace=True)
            testing = pd.merge(latlon_addy,test,how='right',on='ticket_id')
            testing.sort_values(by='ticket_id',inplace=True)

            lat = testing['lat'].mean()
            lon = testing['lon'].mean()

            testing['lat'].fillna(lat,inplace=True)
            testing['lon'].fillna(lon,inplace=True)


            def gradboost():
                X = training.drop(['compliance','merge_addy','disposition_Responsible (Fine Waived) by Deter'],axis=1)
               
                y = training['compliance'].astype(int)

                X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=4)

                clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=3,max_features=4,n_estimators=200).fit(X_train,y_train)

                y_pred = clf.predict_proba(X_test)[:,1]

                series = pd.Series(y_pred,index=X_test['ticket_id'])

                return series
            
            return gradboost()
        
        return geo()

    return wrangle()

series = blight_model(w,x,y,z)

<ipython-input-22-95be39e9842b>:59: FutureWarning: The default value of regex will change from True to False in a future version.
  latlon['merge_addy'] = latlon['merge_addy'].str.replace(',.*','')


In [23]:
series

ticket_id
85537     0.023612
265432    0.031377
69410     0.034590
44576     0.027971
248849    0.017902
            ...   
92324     0.041572
244401    0.027661
267866    0.021484
236723    0.024202
169332    0.050714
Length: 39933, dtype: float64

# Broken Down: (Let's see the details)

### 1.) Import CSV files, perform EDA and Data Wrangling Operations

In [4]:
y = 'readonly/addresses.csv'
z = 'readonly/latlons.csv'
w = 'readonly/train.csv'
x= 'readonly/test.csv'

train = pd.read_csv(w,engine='python')
test = pd.read_csv(x)


train['disposition'].replace(to_replace='Not responsible by City Dismissal',
                            value='Not responsible by Dismissal',inplace=True)
dispos_keep_list= ['Responsible by Default', 'Responsible by Determination',
                   'Responsible by Admission', 'Responsible (Fine Waived) by Deter']

train = train[train['disposition'].isin(dispos_keep_list)].copy()

dispo_dict = {'Responsible (Fine Waived) by Admis':'Responsible by Admission',
              'Responsible - Compl/Adj by Default':'Responsible by Default',
              'Responsible - Compl/Adj by Determi':'Responsible by Determination'}
test['disposition'].replace(dispo_dict,inplace=True)

#test = test[test['disposition'].isin(dispos_keep_list)].copy()

test = test[test.columns[[0,18,25]]].copy()
test = pd.get_dummies(test,columns=['disposition']).copy()

test.head()

,ticket_id,judgment_amount,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination,disposition_Responsible by Dismissal
0,284932,250.0,0,0,1,0,0
1,285362,1130.0,0,0,1,0,0
2,285361,140.0,0,0,1,0,0
3,285338,250.0,0,0,1,0,0
4,285346,140.0,0,0,1,0,0


### 2.) Feature Engineering with One-Hot-Encoding of categorical variables, remove leaky / irrelevant variables

In [5]:
#drop columns not needed after examining each
idx = [1,2,3,6,7,8,9,10,11,12,13,14,15,16,17,28,30,31,32]
train.drop(train.columns[idx],axis=1,inplace=True)

idxes = [1,2,4,5,6,7,8,9,11,12]
train.drop(train.columns[idxes],axis=1,inplace=True)

train = pd.get_dummies(train,columns=['disposition']).copy()

train.drop(train.columns[2],axis=1,inplace=True)

train.head()

,ticket_id,judgment_amount,compliance,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination
0,22056,305.0,0.0,0,0,1,0
1,27586,855.0,1.0,0,0,0,1
5,22046,305.0,0.0,0,0,1,0
6,18738,855.0,0.0,0,0,1,0
7,18735,140.0,0.0,0,0,1,0


### 3.) EDA and Cleaning of other CSV's to create new features on latitude, longitude, and mapping those to addresses in Ann-arbor

In [7]:
addys = pd.read_csv(y)
latlon = pd.read_csv(z)

addr = addys['address'].str.extract('([0-9\sA-Za-z]+),',expand=False)
addys['merge_addy'] = addr

latlon['merge_addy'] = latlon['address'].copy()
latlon['merge_addy'] = latlon['merge_addy'].str.replace(',.*','')

latlon_addy = pd.merge(latlon,addys,how='inner',on='merge_addy')
latlon_addy.drop(['address_x','address_y'],axis=1,inplace=True)
latlon_addy.drop_duplicates(subset='ticket_id',keep='first',inplace=True)

latlon_addy.head()

<ipython-input-7-8f2bdcfb0f72>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  latlon['merge_addy'] = latlon['merge_addy'].str.replace(',.*','')


,lat,lon,merge_addy,ticket_id
0,42.346169,-83.079962,4300 rosa parks blvd,334498
1,42.346169,-83.079962,4300 rosa parks blvd,352642
4,42.394657,-83.194265,14512 sussex,210778
5,42.373779,-82.986228,3456 garland,155846
6,42.373779,-82.986228,3456 garland,155852


### 4.) Merge all features together into a single dataframe for our training data

In [8]:
training = pd.merge(latlon_addy,train,how='inner',on='ticket_id').dropna()
training.sort_values(by='ticket_id',inplace=True)
training.head()

,lat,lon,merge_addy,ticket_id,judgment_amount,compliance,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination
4152,42.383385,-83.072582,601 king,18645,305.0,0.0,0,0,1,0
52888,42.383422,-83.072483,607 king,18646,305.0,0.0,0,0,1,0
45955,42.393440,-83.127929,3005 pasadena,18649,305.0,0.0,0,0,1,0
64040,42.389689,-83.134415,4024 clements,18650,305.0,0.0,0,0,1,0
70721,42.389290,-83.134006,4017 clements,18651,305.0,0.0,0,0,1,0


### 5.) Do the same with our Testing data

In [9]:
testing = pd.merge(latlon_addy,test,how='right',on='ticket_id')
testing.sort_values(by='ticket_id',inplace=True)
testing.head()

,lat,lon,merge_addy,ticket_id,judgment_amount,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination,disposition_Responsible by Dismissal
0,42.407581,-82.986642,10041 roseberry,284932,250.0,0,0,1,0,0
7392,42.321438,-83.096893,4410 uthes,284943,1130.0,0,0,1,0,0
7387,42.306088,-83.113684,7050 lexington,284944,140.0,0,0,1,0,0
7440,42.304815,-83.111996,809 beard,284945,1130.0,0,0,1,0,0
7672,42.333269,-83.123734,6404 morse,284946,140.0,0,0,0,1,0


### 6.) Fill NaN values with mean latitude and longitude values on testing set

In [10]:
lat = testing['lat'].mean()
lon = testing['lon'].mean()

testing['lat'].fillna(lat,inplace=True)
testing['lon'].fillna(lon,inplace=True)
testing.head()

,lat,lon,merge_addy,ticket_id,judgment_amount,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination,disposition_Responsible by Dismissal
0,42.407581,-82.986642,10041 roseberry,284932,250.0,0,0,1,0,0
7392,42.321438,-83.096893,4410 uthes,284943,1130.0,0,0,1,0,0
7387,42.306088,-83.113684,7050 lexington,284944,140.0,0,0,1,0,0
7440,42.304815,-83.111996,809 beard,284945,1130.0,0,0,1,0,0
7672,42.333269,-83.123734,6404 morse,284946,140.0,0,0,0,1,0


### 7.) Train GB Classifier, calculate our AUC score. 

In [24]:
X = training.drop(['compliance','merge_addy','disposition_Responsible (Fine Waived) by Deter'],axis=1)

y = training['compliance'].astype(int)

# test_data = testing.drop(['merge_addy','disposition_Responsible (Fine Waived) by Deter',
#                          'disposition_Responsible by Dismissal'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=4)

clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=3,max_features=4,n_estimators=200).fit(X_train,y_train)

y_pred = clf.predict_proba(X_test)[:,1]

series = pd.Series(y_pred,index=X_test['ticket_id'])

auc = roc_auc_score(y_test,y_pred)
auc

0.8120520271077932

### 8.) Predict_Proba for our Testing dataframe to predict the likilhood of each ticket_id being paid. 

In [28]:
test_data = testing.drop(['merge_addy','disposition_Responsible (Fine Waived) by Deter',
                         'disposition_Responsible by Dismissal'],axis=1)

testing_y_pred = clf.predict_proba(test_data)[:,1]

ticket_payment_probability_by_ticket_id = pd.Series(testing_y_pred,index=test_data['ticket_id'])

ticket_payment_probability_by_ticket_id

ticket_id
284932    0.085864
284943    0.033852
284944    0.143439
284945    0.036075
284946    0.588687
            ...   
376622    0.068426
376623    0.230147
376624    0.978905
376638    0.259239
376698    0.152873
Length: 61001, dtype: float64